In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os, re
from tqdm import tqdm
from scipy.cluster.vq import kmeans

In [3]:
from normalization import process_powered_sheet
from analysis import isDataRise, power_spectrum_with_fitting
from special_items import cluster_k, sampling_frequency, chart_dir, eye_data_path

ModuleNotFoundError: No module named 'normalization'

In [ ]:
temp = process_powered_sheet()
ch_num, data, on_thresh, _ = temp[-1]

In [ ]:
sigma = 9.21163544297e-05
on_thresh = 9*sigma

In [ ]:
kde_raw = np.sign(data['power'].values-on_thresh) #NOTE: skip kde analysis；不做KDE，仅对power后的数据减去阈值后取Sign值；由于历史原因，变量名就没有改。
#kde_raw = kde_anaylsis(data[data.spike > 0]['time'].values[:,np.newaxis], data['time'].values).flatten()
kde_result = pd.DataFrame(np.hstack((data['time'].values[:,np.newaxis], np.sign(kde_raw)[:,np.newaxis])),columns=['time','kde_spike'])

isSeekingRise = 1
bout_time_list = []
print('check data rise:')
for each_pos in tqdm(kde_result[kde_result.kde_spike > 0].index): #REVIEW
    if isDataRise(kde_result, each_pos, step=isSeekingRise):
        bout_time_list.append(kde_result.iloc[each_pos]['time'])
        isSeekingRise = - isSeekingRise

#NB: bout_time_pair: list[(start, end, name)]
bout_time_pair = [(bout_time_list[index*2], bout_time_list[index*2+1], 'bout_%d'%index) for index in range(int(len(bout_time_list)/2))]
bout_time_pair = list(filter(lambda x:x[1]-x[0]>10/sampling_frequency, bout_time_pair)) #REVIEW: XXX: 跳过长度小于0.01，即不足10个数据点的数据段

popt_list, out_name = power_spectrum_with_fitting(data, bout_time_pair, ch_num, isPlot=False) #数据拟合
bout_time_pair = [x for x in bout_time_pair if x[-1] not in out_name]

mean_list = [(data[(data.time >= start) & (data.time <= end)]['raw'].mean(),name) for start, end, name in bout_time_pair]
argument_list = [(popt_list[index][0], popt_list[index][1], mean_list[index][0], mean_list[index][1]) for index in range(len(mean_list))]
exile_name = [x[-1] for x in argument_list if x[1] < -10]      #REVIEW
argument_list = list(filter(lambda x:x[1]>-10, argument_list)) #REVIEW
bout_time_pair = [x for x in bout_time_pair if x[-1] not in exile_name]

#k-means clustering
#NB:codebook: np.array([A, tau, C])
codebook, distortion = kmeans(list(map(lambda x:x[:3],argument_list)),cluster_k) #TODO: modifier other parameters
print('k-means result:', codebook, distortion)

#cluster and delete data
#NB: scatter_result:list[(A, tau, mean, name, genre)]
scatter_result = []
for item in argument_list:
    distance1 = np.sqrt((item[0]-codebook[0][0])**2+(item[1]-codebook[0][1])**2+(item[2]-codebook[0][2])**2)
    distance2 = np.sqrt((item[0]-codebook[1][0])**2+(item[1]-codebook[1][1])**2+(item[2]-codebook[1][2])**2)
    genre = 1 if distance1 > distance2 else 2
    scatter_result.append((item[0], item[1], item[2], item[3], genre))





if ch_num == 1: #CHANGED: 跳过Ch 0通道；仅仅是为了方便分析Ch 1的数据。
    eye = pd.read_csv(eye_data_path) #NOTE:
    for index in eye.index:
        plt.plot(eye.iloc[index].tolist(), np.zeros(eye.iloc[index].shape)-on_thresh*2,c='black',linewidth=2.5)


plt.xlim(data['time'].values[0], data['time'].values[-1])
plt.savefig(os.path.join(chart_dir,'R_raw_CH_%d.png'%ch_num), bbox_inches='tight')

#TODO: 自动分割
for start, end in [(60,500),(500,1000),(1000,1500),(1500,2000)]:#,
                           #(2000,2500),(2500,3000),(3000,3500),(3500,4000),
                           #(4000,4500),(4500,5000),(5000,5500),(5500,6000),
                           #(6000,6500),(6500,7000),(7000,7500),(7500,8000)]:
    plt.xlim(start, end)
    plt.savefig(os.path.join(chart_dir,'R_raw_CH_%d_%d_%d.png'%(ch_num,start,end)), bbox_inches='tight')
plt.close()

print('CH_%d processed'%ch_num)

In [ ]:
#visualization
# SATM
plt.figure(figsize=(30,30))
plt.scatter(x=[x for _,x,_,_,_ in scatter_result ], y=[y for _,_,y,_,_ in scatter_result],s=50, c=['g' if c==1 else 'r' for _,_,_,_,c in scatter_result])#REVIEW , s=[-np.log10(z) for z,_,_ in argument_list])
for _,x,y, in codebook:
    plt.scatter(x=x,y=y,s=100,c='#808080')
#plt.xlim(-3,7)
plt.show()
plt.close('all')

In [ ]:
# R_?->raw, kde, data_rise, chew_bout
plt.figure(figsize=(100,10))
plt.plot(data['time'].values, data['power'].values,'pink')
plt.plot(data['time'].values, np.ones(data['time'].values.shape)*on_thresh, c='#808080')
            #plt.plot(data['time'].values, np.ones(data['time'].values.shape)*off_thresh, 'r'))
            #axes[0].xlim(data['time'].values[0],data['time'].values[-1])
bout_height = data['power'].values.max()

for index in range(len(bout_time_pair)): #scatter_result:list[(A, tau, mean, name, genre)]
    if scatter_result[index][-1] == 1:
        color = 'g'
    else:
        color = 'r'
    target_time = data[(data.time>=bout_time_pair[index][0]) & (data.time<=bout_time_pair[index][1])]['time'].values
    plt.plot(target_time, np.zeros(target_time.shape)-on_thresh,c=color,linewidth=2.5)
if ch_num == 1: #CHANGED: 跳过Ch 0通道；仅仅是为了方便分析Ch 1的数据。
    eye = pd.read_csv(eye_data_path) #NOTE:
    for index in eye.index:
        plt.plot(eye.iloc[index].tolist(), np.zeros(eye.iloc[index].shape)-on_thresh*2,c='black',linewidth=2.5)

#TODO: 自动分割
for start, end in [(60,500)]:
    plt.xlim(start, end)
    plt.show()
plt.close()

print('CH_%d processed'%ch_num)